# Early prediction of Alzheimer's disease and related dementias based on social determinants using TabNet
In this Jupyter Notebook we use social determinants to estimate a composite score, which is associated with Alzheimer's disease and related dementias. We use TabNet [1] to estimate the composite score, where higher score is better, and the maximum possible score is 384. TabNet is a transformer-based neural network tailor made for regression/classification based on tabular data. The data set have a lot of missing values. We use simple imputations using the mean for numerical variables and the most frequent for categorical variables. 
tart!

## Part 1: Packages and frameworks
First we needto oinstall and import some Python packages. In this notebook we will us ePytorc  t oimplement a TabNe model. o interpret the variables and predict the **composite_score**.

In [1]:
%%capture
!pip install pytorch-tabnet

In [2]:
import json
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_squared_error
from pytorch_tabnet.tab_model import TabNetRegressor
from sklearn.preprocessing import LabelEncoder, StandardScaler

## Part 2: Load data
In this part we load and do some initial exploration of the data

In [ ]:
# Load datasets
train_features = pd.read_csv('/your/path/here/train_features.csv') # load train features
train_labels = pd.read_csv('/your/path/here/train_labels.csv') # load train labels
test_features = pd.read_csv('/your/path/here/test_features.csv') # loead test features
submission_format = pd.read_csv('/your/path/here/submission_format.csv') # submission format

In [4]:
print(f"In the training data we have " + str(train_features.shape[1]) + " columns")
print(f"In the test data we have " + str(test_features.shape[1]) + " columns")

In the training data we have 184 columns
In the test data we have 184 columns


In [5]:
print(f"In the training data we have " + str(train_features["uid"].nunique()) + " unique pasients and there are " + str(train_features.shape[0]) + " rows in the training data")
print(f"In the label data we have " + str(train_labels["uid"].nunique()) + " unique pasients and there are " + str(train_labels.shape[0]) + " rows in the training data")

In the training data we have 3276 unique pasients and there are 3276 rows in the training data
In the label data we have 3276 unique pasients and there are 4343 rows in the training data


We see that there are more rows in the label file than we have rows in the train features. This is because we want to estimate the composite score for 2016 (4 years in the future) and 2021 (9 years in the future) for some patients. This is also the cas for the test data:

In [6]:
print(f"In the training data we have " + str(test_features["uid"].nunique()) + " unique pasients and there are " + str(test_features.shape[0]) + " rows in the training data")
print(f"In the label data we have " + str(submission_format["uid"].nunique()) + " unique pasients and there are " + str(submission_format.shape[0]) + " rows in the training data")

In the training data we have 819 unique pasients and there are 819 rows in the training data
In the label data we have 819 unique pasients and there are 1105 rows in the training data


In [7]:
# Count number of patinents with composite score at both 2016 and 2021 in the training data
(train_labels.groupby("uid").count()["year"] == 2).value_counts()

year
False    2209
True     1067
Name: count, dtype: int64

In [8]:
# Count number of patinents we need to predict composite for both 2016 and 2021 in the test data
(submission_format.groupby("uid").count()["year"] == 2).value_counts()

year
False    533
True     286
Name: count, dtype: int64

Exploring missing values

In [9]:
# How many columns have at least one missing value (training data)
(train_features.isna().sum() != 0).value_counts()

True     182
False      2
Name: count, dtype: int64

In [10]:
# How many columns have at least one missing value (test data)
(test_features.isna().sum() != 0).value_counts()

True     182
False      2
Name: count, dtype: int64

In [11]:
round((train_features.isna().sum().sum() / (train_features.shape[0]*train_features.shape[1]))*100,2)

22.45

In [12]:
print("In total there are " + str(round((train_features.isna().sum().sum() / (train_features.shape[0]*train_features.shape[1]))*100,2)) + " % missing values in the training data")

In total there are 22.45 % missing values in the training data


In [13]:
print("In total there are " + str(round((test_features.isna().sum().sum() / (test_features.shape[0]*test_features.shape[1]))*100,2)) + " % missing values in the test data")

In total there are 21.92 % missing values in the test data


## Part 3: Preprocessing
In this part we preprocess the data based on our findings in part 2

In [14]:
# duplicate features where we need to estimate composite score for both 2016 and 2021 (training data)
train_data = train_labels.merge(train_features, on="uid")
train_data["pred_year"] = train_data["year"]-2012

In [15]:
# duplicate features where we need to estimate composite score for both 2016 and 2021 (test data)
aligned_test_features = submission_format[["uid","year"]].merge(test_features, on="uid")
aligned_test_features["pred_year"] = aligned_test_features["year"]-2012

In [16]:
# Separate features and target variable
X = train_data.drop(columns=['uid', 'year', 'composite_score'])
y = train_data['composite_score']

In [17]:
# Handle missing values
num_imputer = SimpleImputer(strategy='mean')
cat_imputer = SimpleImputer(strategy='most_frequent')

In [18]:
# Select numerical and categorical columns
num_cols = X.select_dtypes(include=['float64', 'int64']).columns
cat_cols = X.select_dtypes(include=['object']).columns

In [19]:
# Impute training features
X[num_cols] = num_imputer.fit_transform(X[num_cols])
X[cat_cols] = cat_imputer.fit_transform(X[cat_cols])

In [20]:
# Encode categorical variables
label_encoders = {}
for col in cat_cols:
    le = LabelEncoder()
    X[col] = le.fit_transform(X[col])
    label_encoders[col] = le

In [21]:
# Split data into training and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

## Part 4: Initialize model and start training
Here we initialize our TabNet model and start the training. We continously validate our model on the validation set for each epoch

In [22]:
# Initialize TabNet
tabnet = TabNetRegressor()

# Train TabNet
tabnet.fit(
    X_train.values, np.asarray(y_train).reshape(-1, 1),
    eval_set=[(X_train.values, np.asarray(y_train).reshape(-1, 1)), (X_val.values, np.asarray(y_val).reshape(-1, 1))],
    eval_metric=['rmse'],
    max_epochs=100,
    patience=10,
    batch_size=256,
    virtual_batch_size=128
)

# Predictions and evaluation
predictions = tabnet.predict(X_val.values)
print('Validation RMSE:', np.sqrt(mean_squared_error(np.asarray(y_val).reshape(-1, 1), predictions)))

/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 28359.95433| val_0_rmse: 166.67525| val_1_rmse: 165.28185|  0:00:01s
epoch 1  | loss: 26936.07527| val_0_rmse: 137.5387| val_1_rmse: 136.61888|  0:00:01s
epoch 2  | loss: 24761.41692| val_0_rmse: 147.10145| val_1_rmse: 145.39498|  0:00:02s
epoch 3  | loss: 21441.0284| val_0_rmse: 125.61941| val_1_rmse: 123.78711|  0:00:02s
epoch 4  | loss: 16988.70267| val_0_rmse: 114.22674| val_1_rmse: 112.94602|  0:00:03s
epoch 5  | loss: 11916.67142| val_0_rmse: 64.31498| val_1_rmse: 64.29111|  0:00:03s
epoch 6  | loss: 7146.25139| val_0_rmse: 60.05406| val_1_rmse: 58.37576|  0:00:04s
epoch 7  | loss: 3719.27575| val_0_rmse: 428.86471| val_1_rmse: 428.66333|  0:00:04s
epoch 8  | loss: 2576.53382| val_0_rmse: 381.05859| val_1_rmse: 381.39976|  0:00:05s
epoch 9  | loss: 2373.22388| val_0_rmse: 195.33206| val_1_rmse: 194.99814|  0:00:05s
epoch 10 | loss: 2187.60503| val_0_rmse: 71.73982| val_1_rmse: 66.30959|  0:00:06s
epoch 11 | loss: 2113.75499| val_0_rmse: 61.04225| val_1_rmse: 58.3

/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


Validation RMSE: 58.37576111073713


In [23]:
# Train a final model using hyperparameters (use best epoch from previuos cell) and all development data (train+val)
final_model = TabNetRegressor()

# Train TabNet
final_model.fit(
    X.values, np.asarray(y).reshape(-1, 1),
    eval_set=[(X.values, np.asarray(y).reshape(-1, 1))],
    max_epochs=45,
    batch_size=256,
    virtual_batch_size=128,
    eval_metric=['rmse'],
)

/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/abstract_model.py:82: UserWarning: Device used : cuda
  warnings.warn(f"Device used : {self.device}")


epoch 0  | loss: 27922.88525| val_0_rmse: 264.11093|  0:00:00s
epoch 1  | loss: 26022.98462| val_0_rmse: 156.94842|  0:00:01s
epoch 2  | loss: 22852.02246| val_0_rmse: 143.31492|  0:00:01s
epoch 3  | loss: 17535.14587| val_0_rmse: 116.3263|  0:00:02s
epoch 4  | loss: 11462.24423| val_0_rmse: 87.74657|  0:00:02s
epoch 5  | loss: 5793.24109| val_0_rmse: 62.89397|  0:00:03s
epoch 6  | loss: 2994.51752| val_0_rmse: 61.40496|  0:00:03s
epoch 7  | loss: 2523.89862| val_0_rmse: 57.29636|  0:00:04s
epoch 8  | loss: 2391.05067| val_0_rmse: 60.09168|  0:00:05s
epoch 9  | loss: 2223.51469| val_0_rmse: 60.79312|  0:00:05s
epoch 10 | loss: 2146.12557| val_0_rmse: 59.32807|  0:00:06s
epoch 11 | loss: 2123.29388| val_0_rmse: 60.72531|  0:00:06s
epoch 12 | loss: 1968.39115| val_0_rmse: 56.27555|  0:00:07s
epoch 13 | loss: 1962.16911| val_0_rmse: 51.4107 |  0:00:07s
epoch 14 | loss: 1891.6561| val_0_rmse: 50.1372 |  0:00:08s
epoch 15 | loss: 1874.98175| val_0_rmse: 52.09332|  0:00:09s
epoch 16 | loss: 

/opt/conda/lib/python3.10/site-packages/pytorch_tabnet/callbacks.py:172: UserWarning: Best weights from best epoch are automatically used!
  warnings.warn(wrn_msg)


## Part 5: Inference and submission
Prepare test data for inference, do inference and save the estimations to a submission file 

In [24]:
# Impute missing values in test data
aligned_test_features[num_cols] = num_imputer.transform(aligned_test_features[num_cols])
aligned_test_features[cat_cols] = cat_imputer.transform(aligned_test_features[cat_cols])

# Encode categorical variables in test data
for col in cat_cols:
    aligned_test_features[col] = label_encoders[col].transform(aligned_test_features[col])

# Drop year and id from input features
X_aligned_test = aligned_test_features.drop(columns=['uid','year'])

In [25]:
# Predict on the aligned test dataset
aligned_test_predictions = final_model.predict(X_aligned_test.values)

In [26]:
# Prepare the submission file
submission = submission_format.copy()
submission['composite_score'] = aligned_test_predictions.round().astype(int)

# Save the submission file
submission_file_path = 'submission.csv'
submission.to_csv(submission_file_path, index=False)
print(f"Submission file saved to: {submission_file_path}")

Submission file saved to: submission.csv


## References
[1] [TabNet: Attentive Interpretable Tabular Learning](https://arxiv.org/abs/1908.07442)